# Полный анализ датасета Game of Thrones

## Введение

Этот ноутбук содержит **подробный анализ** датасета персонажей из вселенной "Игры Престолов" (A Song of Ice and Fire). Мы проведем:

1. **Исследовательский анализ данных (EDA)**
2. **Визуализацию данных**
3. **Предобработку и очистку данных**
4. **Feature Engineering**
5. **Построение предсказательных моделей**
6. **Оценку и сравнение моделей**

---

## О датасете

**Датасет Game of Thrones** содержит информацию о персонажах из A Wiki of Ice and Fire. Наша цель - предсказать, кто из персонажей умрет, а кто останется в живых.

### Основные признаки:

- `name` - Имя персонажа
- `Title` - Социальный статус или знатность
- `House` - Дом, к которому принадлежит персонаж
- `Culture` - Социальная группа
- `book1/2/3/4/5` - Появление персонажа в книге
- `Is noble` - Знатность персонажа
- `Age` - Возраст (отсчет: 305 AC)
- `male` - Пол (1 = мужчина, 0 = женщина)
- `dateOfBirth` - Дата рождения
- `Spouse` - Имя супруги/супруга
- `Father` - Имя отца
- `Mother` - Имя матери
- `Heir` - Имя наследника
- `Is married` - Женат/замужем ли персонаж
- `Is spouse alive` - Жив ли супруг/супруга
- `Is mother alive` - Жива ли мать
- `Is heir alive` - Жив ли наследник
- `Is father alive` - Жив ли отец
- `Number dead relations` - Количество умерших родственников
- `Popularity score` - Популярность на вики (количество ссылок)
- `isAlive` - **Целевая переменная** (1 = жив, 0 = мертв)

## 1. Импорт необходимых библиотек

Начнем с импорта всех необходимых библиотек для анализа данных, визуализации и машинного обучения.

In [ ]:
# Основные библиотеки для работы с данными
import pandas as pd
import numpy as np

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка стиля визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Для работы с предупреждениями
import warnings
warnings.filterwarnings('ignore')

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

print("Все библиотеки успешно импортированы!")

## 2. Загрузка данных

Загружаем датасеты train и test. Если у вас нет файлов локально, их можно загрузить с Kaggle.

In [ ]:
# Загрузка данных
# Замените пути на актуальные, если файлы находятся в другом месте

try:
    df_train = pd.read_csv('../../datasets/game_of_thrones_train.csv')
    df_test = pd.read_csv('../../datasets/game_of_thrones_test.csv')
    print("Данные успешно загружены из локальных файлов!")
except FileNotFoundError:
    print("Файлы не найдены. Пожалуйста, загрузите датасет Game of Thrones.")
    print("Вы можете найти его на Kaggle или других источниках.")
    # В качестве демонстрации создадим пустые DataFrame
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()

# Объединяем для анализа (позже разделим)
if not df_train.empty:
    df = df_train.copy()
    df_original = df.copy()
    
    print(f"\nРазмер обучающего датасета: {df_train.shape}")
    print(f"Размер тестового датасета: {df_test.shape}")
    print(f"\nОбщее количество признаков: {df.shape[1]}")

## 3. Первичный осмотр данных

Изучим структуру данных и получим общее представление о датасете.

In [ ]:
# Первые строки датасета
print("Первые 10 строк датасета:")
df.head(10)

In [ ]:
# Информация о типах данных и пропущенных значениях
print("Информация о датасете:")
df.info()

In [ ]:
# Статистическое описание числовых признаков
print("Статистическое описание числовых признаков:")
df.describe()

In [ ]:
# Названия всех колонок
print("Список всех признаков:")
print(df.columns.tolist())

## 4. Анализ пропущенных значений

Пропущенные значения могут существенно влиять на качество моделей. Важно идентифицировать и правильно обработать их.

In [ ]:
# Подсчет пропущенных значений
missing_values = df.isnull().sum()
missing_percent = 100 * df.isnull().sum() / len(df)

missing_df = pd.DataFrame({
    'Признак': missing_values.index,
    'Пропущено значений': missing_values.values,
    'Процент': missing_percent.values
})

missing_df = missing_df[missing_df['Пропущено значений'] > 0].sort_values('Пропущено значений', ascending=False)

print("Пропущенные значения в датасете:")
print(missing_df)

In [ ]:
# Визуализация пропущенных значений
plt.figure(figsize=(14, 8))
sns.heatmap(df.isnull(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Матрица пропущенных значений', fontsize=16, fontweight='bold')
plt.xlabel('Признаки', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 5. Исследовательский анализ данных (EDA)

### 5.1 Анализ целевой переменной (isAlive)

Изучим распределение живых и мертвых персонажей.

In [ ]:
# Проверяем наличие целевой переменной
if 'isAlive' in df.columns:
    # Подсчет выживших
    survival_counts = df['isAlive'].value_counts()
    survival_percent = 100 * df['isAlive'].value_counts(normalize=True)
    
    print("Распределение персонажей:")
    print(f"Мертвых (0): {survival_counts.get(0, 0)} ({survival_percent.get(0, 0):.2f}%)")
    print(f"Живых (1): {survival_counts.get(1, 0)} ({survival_percent.get(1, 0):.2f}%)")
    print(f"\nОбщий уровень выживаемости: {survival_percent.get(1, 0):.2f}%")
else:
    print("Целевая переменная 'isAlive' не найдена в датасете")

In [ ]:
# Визуализация
if 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Столбчатая диаграмма
    sns.countplot(data=df, x='isAlive', ax=axes[0], palette='Set2')
    axes[0].set_title('Количество живых и мертвых персонажей', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Статус (0 = Мертв, 1 = Жив)', fontsize=12)
    axes[0].set_ylabel('Количество', fontsize=12)
    
    # Добавляем значения на столбцы
    for container in axes[0].containers:
        axes[0].bar_label(container)
    
    # Круговая диаграмма
    axes[1].pie(survival_counts, labels=['Мертвых', 'Живых'], autopct='%1.1f%%', 
                startangle=90, colors=['#ff9999', '#66b3ff'])
    axes[1].set_title('Процентное соотношение', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

### 5.2 Анализ выживаемости по полу

Проверим, влияет ли пол персонажа на выживаемость в мире Вестероса.

In [ ]:
# Выживаемость по полу
if 'male' in df.columns and 'isAlive' in df.columns:
    sex_survival = pd.crosstab(df['male'], df['isAlive'], normalize='index') * 100
    print("Процент выживаемости по полу:")
    print(sex_survival)
    print("\n0 = Женщины, 1 = Мужчины")

In [ ]:
# Визуализация
if 'male' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Сгруппированная столбчатая диаграмма
    sex_counts = pd.crosstab(df['male'], df['isAlive'])
    sex_counts.plot(kind='bar', ax=axes[0], color=['#ff6b6b', '#4ecdc4'])
    axes[0].set_title('Выживаемость по полу', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Пол (0 = Женщины, 1 = Мужчины)', fontsize=12)
    axes[0].set_ylabel('Количество', fontsize=12)
    axes[0].set_xticklabels(['Женщины', 'Мужчины'], rotation=0)
    axes[0].legend(['Мертвых', 'Живых'])
    axes[0].grid(axis='y', alpha=0.3)
    
    # Процентная диаграмма
    sex_survival.plot(kind='bar', ax=axes[1], stacked=True, color=['#ff6b6b', '#4ecdc4'])
    axes[1].set_title('Процент выживаемости по полу', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Пол', fontsize=12)
    axes[1].set_ylabel('Процент', fontsize=12)
    axes[1].set_xticklabels(['Женщины', 'Мужчины'], rotation=0)
    axes[1].legend(['Мертвых', 'Живых'])
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 5.3 Анализ выживаемости по знатности

В мире GoT знатность может играть важную роль в выживаемости персонажа.

In [ ]:
# Выживаемость по знатности
if 'isNoble' in df.columns and 'isAlive' in df.columns:
    noble_survival = pd.crosstab(df['isNoble'], df['isAlive'], normalize='index') * 100
    print("Процент выживаемости по знатности:")
    print(noble_survival)
    print("\n0 = Простолюдины, 1 = Знать")

In [ ]:
# Визуализация
if 'isNoble' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Столбчатая диаграмма
    noble_counts = pd.crosstab(df['isNoble'], df['isAlive'])
    noble_counts.plot(kind='bar', ax=axes[0], color=['#e74c3c', '#2ecc71'])
    axes[0].set_title('Выживаемость по знатности', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Знатность (0 = Простолюдин, 1 = Знатный)', fontsize=12)
    axes[0].set_ylabel('Количество', fontsize=12)
    axes[0].set_xticklabels(['Простолюдины', 'Знать'], rotation=0)
    axes[0].legend(['Мертвых', 'Живых'])
    axes[0].grid(axis='y', alpha=0.3)
    
    # Процентная диаграмма
    noble_survival.plot(kind='bar', ax=axes[1], stacked=True, color=['#e74c3c', '#2ecc71'])
    axes[1].set_title('Процент выживаемости по знатности', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Знатность', fontsize=12)
    axes[1].set_ylabel('Процент', fontsize=12)
    axes[1].set_xticklabels(['Простолюдины', 'Знать'], rotation=0)
    axes[1].legend(['Мертвых', 'Живых'])
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 5.4 Анализ возраста

Изучим распределение возраста персонажей и его влияние на выживаемость.

In [ ]:
# Статистика по возрасту
if 'age' in df.columns:
    print("Статистика по возрасту:")
    print(df['age'].describe())
    print(f"\nСредний возраст: {df['age'].mean():.2f} лет")
    print(f"Медианный возраст: {df['age'].median():.2f} лет")

In [ ]:
# Визуализация распределения возраста
if 'age' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Гистограмма возраста
    axes[0, 0].hist(df['age'].dropna(), bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 0].set_title('Распределение возраста персонажей', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Возраст', fontsize=12)
    axes[0, 0].set_ylabel('Частота', fontsize=12)
    axes[0, 0].axvline(df['age'].mean(), color='red', linestyle='--', label=f'Среднее: {df["age"].mean():.1f}')
    axes[0, 0].axvline(df['age'].median(), color='green', linestyle='--', label=f'Медиана: {df["age"].median():.1f}')
    axes[0, 0].legend()
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # 2. Распределение возраста по выживаемости
    df[df['isAlive']==1]['age'].hist(bins=30, ax=axes[0, 1], color='green', alpha=0.6, label='Живые')
    df[df['isAlive']==0]['age'].hist(bins=30, ax=axes[0, 1], color='red', alpha=0.6, label='Мертвые')
    axes[0, 1].set_title('Распределение возраста по статусу', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Возраст', fontsize=12)
    axes[0, 1].set_ylabel('Частота', fontsize=12)
    axes[0, 1].legend()
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # 3. Box plot возраста по выживаемости
    sns.boxplot(data=df, x='isAlive', y='age', ax=axes[1, 0], palette='Set2')
    axes[1, 0].set_title('Возраст по статусу (Box Plot)', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Статус (0 = Мертв, 1 = Жив)', fontsize=12)
    axes[1, 0].set_ylabel('Возраст', fontsize=12)
    
    # 4. Violin plot возраста по выживаемости и полу
    if 'male' in df.columns:
        sns.violinplot(data=df, x='isAlive', y='age', hue='male', split=True, ax=axes[1, 1], palette='muted')
        axes[1, 1].set_title('Возраст по статусу и полу (Violin Plot)', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Статус (0 = Мертв, 1 = Жив)', fontsize=12)
        axes[1, 1].set_ylabel('Возраст', fontsize=12)
        axes[1, 1].legend(title='Пол', labels=['Женщины', 'Мужчины'])
    
    plt.tight_layout()
    plt.show()

### 5.5 Анализ появлений в книгах

Персонажи появляются в разных книгах серии. Посмотрим, как это связано с выживаемостью.

In [ ]:
# Подсчет появлений в книгах
book_cols = ['book1', 'book2', 'book3', 'book4', 'book5']
available_book_cols = [col for col in book_cols if col in df.columns]

if available_book_cols:
    # Создаем признак общего количества книг
    df['total_books'] = df[available_book_cols].sum(axis=1)
    
    print("Распределение по количеству книг:")
    print(df['total_books'].value_counts().sort_index())
    
    # Выживаемость по количеству книг
    if 'isAlive' in df.columns:
        book_survival = df.groupby('total_books')['isAlive'].agg(['mean', 'count'])
        book_survival['mean'] = book_survival['mean'] * 100
        print("\nВыживаемость по количеству книг:")
        print(book_survival)

In [ ]:
# Визуализация
if 'total_books' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Распределение по количеству книг
    df['total_books'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='teal', alpha=0.7)
    axes[0].set_title('Распределение персонажей по количеству книг', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Количество книг', fontsize=12)
    axes[0].set_ylabel('Количество персонажей', fontsize=12)
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Выживаемость по количеству книг
    book_survival['mean'].plot(kind='bar', ax=axes[1], color='orange', alpha=0.7)
    axes[1].set_title('Процент выживших по количеству книг', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Количество книг', fontsize=12)
    axes[1].set_ylabel('Процент живых (%)', fontsize=12)
    axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
    axes[1].axhline(y=df['isAlive'].mean()*100, color='r', linestyle='--', label='Средняя выживаемость')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 5.6 Анализ популярности

Популярность на вики может указывать на важность персонажа в сюжете.

In [ ]:
# Статистика по популярности
if 'popularity' in df.columns:
    print("Статистика по популярности:")
    print(df['popularity'].describe())
    print(f"\nСредняя популярность: {df['popularity'].mean():.2f}")
    print(f"Медианная популярность: {df['popularity'].median():.2f}")

In [ ]:
# Визуализация популярности
if 'popularity' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Распределение популярности
    axes[0].hist(df['popularity'].dropna(), bins=50, color='purple', edgecolor='black', alpha=0.7)
    axes[0].set_title('Распределение популярности персонажей', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Популярность (количество ссылок)', fontsize=12)
    axes[0].set_ylabel('Частота', fontsize=12)
    axes[0].set_xlim(0, df['popularity'].quantile(0.95))  # Ограничение для лучшей видимости
    axes[0].grid(axis='y', alpha=0.3)
    
    # Box plot популярности по выживаемости
    sns.boxplot(data=df, x='isAlive', y='popularity', ax=axes[1], palette='Set3')
    axes[1].set_title('Популярность по статусу персонажа', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Статус (0 = Мертв, 1 = Жив)', fontsize=12)
    axes[1].set_ylabel('Популярность', fontsize=12)
    axes[1].set_ylim(0, df['popularity'].quantile(0.95))
    
    plt.tight_layout()
    plt.show()

### 5.7 Анализ умерших родственников

Количество умерших родственников может быть важным предиктором.

In [ ]:
# Статистика по умершим родственникам
if 'numDeadRelations' in df.columns:
    print("Статистика по количеству умерших родственников:")
    print(df['numDeadRelations'].describe())
    
    if 'isAlive' in df.columns:
        print("\nСреднее количество умерших родственников:")
        print(f"У живых: {df[df['isAlive']==1]['numDeadRelations'].mean():.2f}")
        print(f"У мертвых: {df[df['isAlive']==0]['numDeadRelations'].mean():.2f}")

In [ ]:
# Визуализация
if 'numDeadRelations' in df.columns and 'isAlive' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Распределение умерших родственников
    axes[0].hist(df['numDeadRelations'].dropna(), bins=30, color='crimson', edgecolor='black', alpha=0.7)
    axes[0].set_title('Распределение умерших родственников', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Количество умерших родственников', fontsize=12)
    axes[0].set_ylabel('Частота', fontsize=12)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Box plot по выживаемости
    sns.boxplot(data=df, x='isAlive', y='numDeadRelations', ax=axes[1], palette='Reds')
    axes[1].set_title('Умершие родственники по статусу персонажа', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Статус (0 = Мертв, 1 = Жив)', fontsize=12)
    axes[1].set_ylabel('Количество умерших родственников', fontsize=12)
    
    plt.tight_layout()
    plt.show()

### 5.8 Корреляционный анализ

Изучим корреляции между числовыми признаками.

In [ ]:
# Выбираем числовые признаки для корреляции
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 0:
    correlation_matrix = df[numeric_cols].corr()
    
    if 'isAlive' in correlation_matrix.columns:
        print("Корреляция признаков с выживаемостью:")
        print(correlation_matrix['isAlive'].sort_values(ascending=False))

In [ ]:
# Визуализация корреляционной матрицы
if len(numeric_cols) > 0:
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Корреляционная матрица признаков', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 6. Предобработка данных

### 6.1 Обработка пропущенных значений

In [ ]:
# Создаем копию для предобработки
df_processed = df.copy()

# Заполнение пропущенных значений в числовых признаках медианой
numeric_features = df_processed.select_dtypes(include=[np.number]).columns

for col in numeric_features:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna(df_processed[col].median(), inplace=True)

# Заполнение категориальных признаков
categorical_features = df_processed.select_dtypes(include=['object']).columns

for col in categorical_features:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col].fillna('Unknown', inplace=True)

print("Пропущенные значения после обработки:")
print(df_processed.isnull().sum().sum())

### 6.2 Feature Engineering

Создадим дополнительные признаки для улучшения предсказательной способности модели.

In [ ]:
# 1. Признак количества книг (если еще не создан)
book_cols = ['book1', 'book2', 'book3', 'book4', 'book5']
available_book_cols = [col for col in book_cols if col in df_processed.columns]

if available_book_cols and 'total_books' not in df_processed.columns:
    df_processed['total_books'] = df_processed[available_book_cols].sum(axis=1)

# 2. Признак наличия живых родственников
alive_relations_cols = ['isSpouseAlive', 'isMotherAlive', 'isFatherAlive', 'isHeirAlive']
available_alive_cols = [col for col in alive_relations_cols if col in df_processed.columns]

if available_alive_cols:
    df_processed['num_alive_relations'] = df_processed[available_alive_cols].sum(axis=1)

# 3. Признак наличия семьи
family_cols = ['isMarried', 'isSpouseAlive', 'isMotherAlive', 'isFatherAlive', 'isHeirAlive']
available_family_cols = [col for col in family_cols if col in df_processed.columns]

if available_family_cols:
    df_processed['has_family'] = (df_processed[available_family_cols].sum(axis=1) > 0).astype(int)

# 4. Категории популярности (если есть признак)
if 'popularity' in df_processed.columns:
    df_processed['popularity_category'] = pd.qcut(df_processed['popularity'], 
                                                   q=4, 
                                                   labels=['Low', 'Medium', 'High', 'Very High'],
                                                   duplicates='drop')

# 5. Возрастные группы (если есть признак)
if 'age' in df_processed.columns:
    df_processed['age_group'] = pd.cut(df_processed['age'], 
                                        bins=[0, 18, 30, 50, 100],
                                        labels=['Young', 'Adult', 'Middle', 'Senior'])

print("Новые признаки созданы!")
print(f"\nТекущее количество признаков: {df_processed.shape[1]}")

### 6.3 Кодирование категориальных признаков

In [ ]:
# Создаем копию для моделирования
df_model = df_processed.copy()

# Удаляем признаки, которые не будем использовать
cols_to_drop = ['name', 'title', 'culture', 'dateOfBirth', 'mother', 'father', 'heir', 'spouse']
cols_to_drop = [col for col in cols_to_drop if col in df_model.columns]

df_model.drop(cols_to_drop, axis=1, inplace=True)

# One-Hot Encoding для категориальных признаков
categorical_features = ['house', 'popularity_category', 'age_group']
categorical_features = [col for col in categorical_features if col in df_model.columns]

for feature in categorical_features:
    # Ограничиваем количество уникальных значений
    if df_model[feature].nunique() > 20:
        # Берем топ-10 категорий, остальные в 'Other'
        top_categories = df_model[feature].value_counts().head(10).index
        df_model[feature] = df_model[feature].apply(lambda x: x if x in top_categories else 'Other')
    
    dummies = pd.get_dummies(df_model[feature], prefix=feature, drop_first=True)
    df_model = pd.concat([df_model, dummies], axis=1)
    df_model.drop(feature, axis=1, inplace=True)

print("Финальный набор признаков:")
print(f"Количество признаков: {df_model.shape[1]}")
print(f"\nПризнаки: {df_model.columns.tolist()}")

## 7. Подготовка данных для моделирования

Разделим данные на обучающую и тестовую выборки.

In [ ]:
# Проверяем наличие целевой переменной
if 'isAlive' in df_model.columns:
    # Разделение на признаки (X) и целевую переменную (y)
    X = df_model.drop('isAlive', axis=1)
    y = df_model['isAlive']
    
    # Разделение на обучающую и тестовую выборки (80/20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"Размер обучающей выборки: {X_train.shape}")
    print(f"Размер тестовой выборки: {X_test.shape}")
    print(f"\nРаспределение классов в обучающей выборке:")
    print(y_train.value_counts(normalize=True))
    print(f"\nРаспределение классов в тестовой выборке:")
    print(y_test.value_counts(normalize=True))
else:
    print("Целевая переменная 'isAlive' не найдена!")

### 7.1 Масштабирование признаков

In [ ]:
if 'isAlive' in df_model.columns:
    # Инициализация scaler
    scaler = StandardScaler()
    
    # Масштабирование
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Преобразование обратно в DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    print("Масштабирование признаков завершено!")

## 8. Построение моделей машинного обучения

Обучим несколько различных моделей и сравним их производительность.

### 8.1 Логистическая регрессия

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    log_reg = LogisticRegression(random_state=42, max_iter=1000)
    log_reg.fit(X_train_scaled, y_train)
    
    # Предсказания
    y_pred_log = log_reg.predict(X_test_scaled)
    y_pred_proba_log = log_reg.predict_proba(X_test_scaled)[:, 1]
    
    # Оценка
    acc_log = accuracy_score(y_test, y_pred_log)
    print(f"Точность логистической регрессии: {acc_log:.4f}")
    
    # Кросс-валидация
    cv_scores_log = cross_val_score(log_reg, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_log.mean():.4f} (+/- {cv_scores_log.std():.4f})")

### 8.2 Дерево решений

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    dt = DecisionTreeClassifier(random_state=42, max_depth=10)
    dt.fit(X_train, y_train)
    
    # Предсказания
    y_pred_dt = dt.predict(X_test)
    y_pred_proba_dt = dt.predict_proba(X_test)[:, 1]
    
    # Оценка
    acc_dt = accuracy_score(y_test, y_pred_dt)
    print(f"Точность дерева решений: {acc_dt:.4f}")
    
    # Кросс-валидация
    cv_scores_dt = cross_val_score(dt, X_train, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_dt.mean():.4f} (+/- {cv_scores_dt.std():.4f})")

### 8.3 Random Forest

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15)
    rf.fit(X_train, y_train)
    
    # Предсказания
    y_pred_rf = rf.predict(X_test)
    y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
    
    # Оценка
    acc_rf = accuracy_score(y_test, y_pred_rf)
    print(f"Точность Random Forest: {acc_rf:.4f}")
    
    # Кросс-валидация
    cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

### 8.4 Gradient Boosting

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    gb = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5, learning_rate=0.1)
    gb.fit(X_train, y_train)
    
    # Предсказания
    y_pred_gb = gb.predict(X_test)
    y_pred_proba_gb = gb.predict_proba(X_test)[:, 1]
    
    # Оценка
    acc_gb = accuracy_score(y_test, y_pred_gb)
    print(f"Точность Gradient Boosting: {acc_gb:.4f}")
    
    # Кросс-валидация
    cv_scores_gb = cross_val_score(gb, X_train, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_gb.mean():.4f} (+/- {cv_scores_gb.std():.4f})")

### 8.5 Support Vector Machine (SVM)

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    svm = SVC(kernel='rbf', random_state=42, probability=True)
    svm.fit(X_train_scaled, y_train)
    
    # Предсказания
    y_pred_svm = svm.predict(X_test_scaled)
    y_pred_proba_svm = svm.predict_proba(X_test_scaled)[:, 1]
    
    # Оценка
    acc_svm = accuracy_score(y_test, y_pred_svm)
    print(f"Точность SVM: {acc_svm:.4f}")
    
    # Кросс-валидация
    cv_scores_svm = cross_val_score(svm, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_svm.mean():.4f} (+/- {cv_scores_svm.std():.4f})")

### 8.6 K-Nearest Neighbors (KNN)

In [ ]:
if 'isAlive' in df_model.columns:
    # Обучение модели
    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(X_train_scaled, y_train)
    
    # Предсказания
    y_pred_knn = knn.predict(X_test_scaled)
    y_pred_proba_knn = knn.predict_proba(X_test_scaled)[:, 1]
    
    # Оценка
    acc_knn = accuracy_score(y_test, y_pred_knn)
    print(f"Точность KNN: {acc_knn:.4f}")
    
    # Кросс-валидация
    cv_scores_knn = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f"Средняя точность на кросс-валидации: {cv_scores_knn.mean():.4f} (+/- {cv_scores_knn.std():.4f})")

## 9. Сравнение моделей

Сравним производительность всех моделей.

In [ ]:
if 'isAlive' in df_model.columns:
    # Создаем DataFrame с результатами
    models_comparison = pd.DataFrame({
        'Модель': ['Logistic Regression', 'Decision Tree', 'Random Forest', 
                   'Gradient Boosting', 'SVM', 'KNN'],
        'Точность': [acc_log, acc_dt, acc_rf, acc_gb, acc_svm, acc_knn],
        'CV Средняя': [cv_scores_log.mean(), cv_scores_dt.mean(), cv_scores_rf.mean(),
                       cv_scores_gb.mean(), cv_scores_svm.mean(), cv_scores_knn.mean()],
        'CV Std': [cv_scores_log.std(), cv_scores_dt.std(), cv_scores_rf.std(),
                   cv_scores_gb.std(), cv_scores_svm.std(), cv_scores_knn.std()]
    })
    
    models_comparison = models_comparison.sort_values('Точность', ascending=False)
    print("Сравнение моделей:")
    print(models_comparison.to_string(index=False))

In [ ]:
# Визуализация сравнения
if 'isAlive' in df_model.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # График точности
    colors = plt.cm.viridis(np.linspace(0, 1, len(models_comparison)))
    bars1 = axes[0].barh(models_comparison['Модель'], models_comparison['Точность'], color=colors)
    axes[0].set_xlabel('Точность', fontsize=12)
    axes[0].set_title('Сравнение точности моделей', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    
    # Добавляем значения
    for i, bar in enumerate(bars1):
        width = bar.get_width()
        axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                    f'{width:.4f}', ha='left', va='center', fontsize=10)
    
    # График кросс-валидации
    axes[1].barh(models_comparison['Модель'], models_comparison['CV Средняя'], 
                 xerr=models_comparison['CV Std'], color=colors, capsize=5)
    axes[1].set_xlabel('Средняя точность (CV)', fontsize=12)
    axes[1].set_title('Точность на кросс-валидации', fontsize=14, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 10. Детальный анализ лучшей модели

Выберем лучшую модель и проведем детальный анализ.

In [ ]:
if 'isAlive' in df_model.columns:
    # Выбираем Random Forest или Gradient Boosting как лучшую модель
    best_model = rf  # или gb, в зависимости от результатов
    y_pred_best = y_pred_rf
    y_pred_proba_best = y_pred_proba_rf
    
    print("Детальный отчет по классификации:")
    print(classification_report(y_test, y_pred_best, target_names=['Мертв', 'Жив']))

### 10.1 Матрица ошибок

In [ ]:
if 'isAlive' in df_model.columns:
    # Матрица ошибок
    cm = confusion_matrix(y_test, y_pred_best)
    
    # Визуализация
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
                xticklabels=['Мертв', 'Жив'],
                yticklabels=['Мертв', 'Жив'])
    plt.title('Матрица ошибок (Confusion Matrix)', fontsize=16, fontweight='bold')
    plt.ylabel('Истинное значение', fontsize=12)
    plt.xlabel('Предсказанное значение', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # Статистика
    tn, fp, fn, tp = cm.ravel()
    print(f"\nДетальная статистика:")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    print(f"True Positives (TP): {tp}")
    print(f"\nAccuracy: {(tp + tn) / (tp + tn + fp + fn):.4f}")
    print(f"Precision: {tp / (tp + fp):.4f}")
    print(f"Recall: {tp / (tp + fn):.4f}")

### 10.2 ROC-кривая и AUC

In [ ]:
if 'isAlive' in df_model.columns:
    # ROC-кривые для всех моделей
    models_roc = [
        ('Logistic Regression', y_pred_proba_log),
        ('Decision Tree', y_pred_proba_dt),
        ('Random Forest', y_pred_proba_rf),
        ('Gradient Boosting', y_pred_proba_gb),
        ('SVM', y_pred_proba_svm),
        ('KNN', y_pred_proba_knn)
    ]
    
    plt.figure(figsize=(10, 8))
    
    for model_name, y_proba in models_roc:
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')
    
    # Диагональная линия
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Случайный классификатор (AUC = 0.5)')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC-кривые для всех моделей', fontsize=16, fontweight='bold')
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

### 10.3 Важность признаков

In [ ]:
if 'isAlive' in df_model.columns:
    # Важность признаков
    feature_importance = pd.DataFrame({
        'Признак': X_train.columns,
        'Важность': rf.feature_importances_
    }).sort_values('Важность', ascending=False)
    
    print("Топ-20 наиболее важных признаков:")
    print(feature_importance.head(20).to_string(index=False))

In [ ]:
if 'isAlive' in df_model.columns:
    # Визуализация
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    colors = plt.cm.viridis(np.linspace(0, 1, len(top_features)))
    
    plt.barh(range(len(top_features)), top_features['Важность'], color=colors)
    plt.yticks(range(len(top_features)), top_features['Признак'])
    plt.xlabel('Важность признака', fontsize=12)
    plt.title('Топ-20 наиболее важных признаков (Random Forest)', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 11. Выводы и заключение

### Основные выводы из анализа:

1. **Выживаемость в GoT**: В мире "Игры Престолов" выживаемость зависит от множества факторов, включая знатность, политические связи и появления в книгах.

2. **Важные факторы**:
   - Популярность персонажа (более популярные персонажи чаще выживают)
   - Количество умерших родственников (индикатор опасности)
   - Появления в книгах (чем больше появлений, тем выше шанс выжить)
   - Знатность и принадлежность к дому
   - Наличие живых родственников

3. **Производительность моделей**: Ансамблевые методы (Random Forest и Gradient Boosting) показывают лучшие результаты.

4. **Интересные паттерны**:
   - Персонажи с большим количеством мертвых родственников имеют более высокий риск смерти
   - Популярные персонажи (главные герои) имеют лучшие шансы на выживание
   - Знатность не всегда гарантирует выживание в жестоком мире Вестероса

### Рекомендации для улучшения:

1. Использовать более продвинутые методы (XGBoost, LightGBM, CatBoost)
2. Провести более глубокий анализ текстовых признаков (титулы, имена домов)
3. Применить техники работы с несбалансированными классами
4. Создать дополнительные признаки на основе семейных связей
5. Использовать SHAP для интерпретации моделей

## 12. Сохранение модели

In [ ]:
import pickle

if 'isAlive' in df_model.columns:
    # Сохранение модели
    with open('../../models/best_got_model.pkl', 'wb') as file:
        pickle.dump(best_model, file)
    
    # Сохранение scaler
    with open('../../models/got_scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)
    
    print("Модель и scaler успешно сохранены!")
    print("Файлы: best_got_model.pkl, got_scaler.pkl")

---

## Конец анализа

Этот ноутбук продемонстрировал полный цикл анализа датасета Game of Thrones:
- Исследовательский анализ данных
- Визуализацию и понимание паттернов
- Предобработку и feature engineering
- Построение и сравнение моделей
- Интерпретацию результатов

**Valar Morghulis** - "Все люди смертны", но наши модели могут предсказать, кто выживет! 🐉

---

*Примечание: Датасет использован исключительно в образовательных целях. "Game of Thrones" и "A Song of Ice and Fire" являются собственностью George R.R. Martin.*